# Step0: Install environment

In [3]:
import vapoursynth as vs
import havsfunc as haf
import mvsfunc as mvf
from vapoursynth import core
import awsmfunc as awsf

# Step1: Select BDMV Largest M2TS and its MLPS

In [ ]:
x265_filename=r"C:\Users\heber\Music\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.x265.DTS-louco@PTHOME\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.x265.DTS-louco@PTHOME.mkv"

# Step2: BDMV to MKV

# Step3: Transcode

In [16]:
import os, pyperclip, re, subprocess
from pymediainfo import MediaInfo
from datetime import datetime
from utils import *
crf=18.0
qcomp=0.7
aq_mode=3
aq_s=0.7
bitdepth=8
newline="\r\n"
remotecwd="~/compressor"
remotehost="amd"
PROCUCT_DEST_DIR=r"C:\Users\heber\Music"
MOVIE=r"C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv"

## get remux capture and crop black borders

In [17]:
get_captures("REMUX", MOVIE)

Capture REMUX


## H264: Test params

In [12]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

args=x264_stmt.replace(" --","\n --").split("\n")[1:]
args=sorted(args)
# print("\n".join(["x264 参数："]+args))

remote_x264stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat|{x264_stmt} 2>&1 | tee x264.log"{newline}'

print("==========\n远程执行编码 x264:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=YES{newline}'
cmd+=remote_x264stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
# cmd+=f'scp {remotehost}:~/264.mkv D:\\BT\\samples\\264.mkv{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

远程执行编码 x264:
set MOVIE=C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
set TEST=YES
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat|x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 18.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log"
scp amd:~/264.mkv C:\Users\heber\Documents\Compressor\264.mkv
pause



## H264&H265

In [2]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"


remote_stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T4|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null"{newline}'


print("==========\n远程执行编码 x264+x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd=f'set TEST=NO{newline}'
cmd+=remote_stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:~/265.mkv {os. getcwd()}\\265.mkv{newline}'

cmd+=f'ssh {remotehost} "cat x264.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x264.log & '
cmd+=f'ssh {remotehost} "cat x265.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x265.log{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

远程执行编码 x264+x265:
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T4|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat| tee >(x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 19.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log) >(x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf 19.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 --qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-op

# H265 4K

In [2]:
x265_stmt=f'x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --psy-rdoq 1 --psy-rd 2  --bframes 11 \
--ctu 64 --min-cu-size 8 --limit-tu 0 --tu-intra-depth 4 --tu-inter-depth 4 \
--me 3 --subme 6 --merange 48 --analyze-src-pics \
--max-merge 5 --early-skip --b-intra \
--no-open-gop --min-keyint 1 --keyint 240 --ref 6 --fades --b-adapt 2 \
--no-strong-intra-smoothing --cbqpoffs -1 --crqpoffs -3 \
--rdoq-level 2 --nr-inter 20 --nr-intra 10 \
--qg-size 32 \
--rd 4 --no-limit-modes --limit-refs 1 --no-rect --rc-lookahead 72 --tskip-fast \
--rdpenalty 0 --no-splitrd-skip \
--deblock -3:-3 --nr-inter 20 --nr-intra 10 \
--no-sao --no-sao-non-deblock --selective-sao 0 \
--no-opt-qp-pps --no-opt-ref-list-length-pps --repeat-headers \
--no-temporal-layers --no-amp --pools + \
--vbv-bufsize 160000 --vbv-maxrate 150000 --lookahead-slices 5 --input-depth 10 \
--hdr10 --hdr10-opt --colormatrix bt2020nc --colorprim bt2020 --transfer smpte2084 \
--master-display \'G(13250,34500)B(7500,3000)R(34000,16000)WP(15635,16450)L(10000000,0)\' --output 265.mkv -'
# --max-cll 922,239 
'''
BT.2020 	G(8500,39850)B(6550,2300)R(35400,14600)WP(15635,16450)L(40000000,50)
Display P3    G(13250,34500)B(7500,3000)R(34000,16000)WP(15635,16450)L(40000000,50)
'''

# with open("x265.sh","w") as f:
#     f.write(f'zstdcat ~/yrm/src|{x265_stmt} 2>&1 | tee x265.log')
# remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T8|ssh {remotehost} "cat>~/y4m/src"'
# remote_convertcmd=f'scp {os.getcwd()}\\x265.sh {remotehost}:{remotecwd}'
    
remote_stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T4|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat| tee \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null"{newline}'


print("==========\n远程执行编码 x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=NO{newline}'
cmd+=remote_stmt
cmd+=f'scp {remotehost}:~/265.mkv {os. getcwd()}\\265.mkv{newline}'

cmd+=f'ssh {remotehost} "cat x265.log|sed \'s/.*\\r//g\' " >{os. getcwd()}\\x265.log{newline}'

pyperclip.copy(cmd)
print(cmd)

远程执行编码 x265:
set MOVIE=C:\Users\heber\Music\Get.Out.2017.UHD.Blu-Ray.2160p.REMUX.DTS-louco@PTH.mkv
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T4|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat| tee >(x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf 18.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --psy-rdoq 1 --psy-rd 2  --bframes 11 --ctu 64 --min-cu-size 8 --limit-tu 0 --tu-intra-depth 4 --tu-inter-depth 4 --me 3 --subme 6 --merange 48 --analyze-src-pics --max-merge 5 --early-skip --b-intra --no-open-gop --min-keyint 1 --keyint 240 --ref 6 --fades --b-adapt 2 --no-strong-intra-smoothing --cbqpoffs -1 --crqpoffs -3 --rdoq-level 2 --nr-inter 20 --nr-intra 10 --qg-size 32 --rd 4 --no-limit-modes --limit-refs 1 --no-rect --rc-lookahead 72 --tskip-fast --rdpenalty 0 --no-splitrd-skip --deblock -3:-3 --nr-inter 20 --nr-intra 10 --no-sao --no-sao-non-debloc

## H264 & H265 offline

In [5]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"

remote_stmt=f'zstdcat ~/yrm/src| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null{newline}'

with open("x26x.sh","w") as f:
    f.write(remote_stmt)
remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T8|ssh {remotehost} "cat>~/y4m/src"'
remote_convertcmd=f'scp {os.getcwd()}\\x26x.sh {remotehost}:{remotecwd}'

print("==========\n远程执行编码 x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=NO{newline}'
cmd+=remote_x265stmt+newline
cmd+=remote_convertcmd+newline
cmd+=f'pause{newline}'
cmd+=f'scp {remotehost}:{remotecwd}/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:{remotecwd}/265.mkv {os. getcwd()}\\265.mkv{newline}'

pyperclip.copy(cmd)
print(cmd)

远程执行编码 x265:
set MOVIE=C:\Users\heber\Music\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.REMUX.HEVC.DTS-MA.7.1-PTHOME\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.REMUX.HEVC.DTS-MA.7.1-PTHOME.mkv
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T8|ssh ubuntu20local "cat>~/y4m/src"
scp C:\Users\heber\Documents\Compressor\x26x.sh ubuntu20local:~/compressor
pause
scp ubuntu20local:~/compressor/265.mkv C:\Users\heber\Documents\Compressor\265.mkv



# Step4: Remux MKV
## x264

In [23]:
# title="".join(re.findall(r'([^\\]*\d{4}p)', MOVIE))
# NEWMOVIE=f'{os.getcwd()}\\{title}.BluRay.x264.DTS-PTH.mkv'
x264_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX.AVC", "x264")
x264_foldername=PROCUCT_DEST_DIR+"\\"+x264_filename.replace(".mkv",'')
x264_fullpath=f'{x264_foldername}\\{x264_filename}'
print(x264_filename)
cmd=f'mkdir {x264_foldername} ^& '
cmd+=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\264.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\264.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome.mkv
即将执行: 
   [剩] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
 + [视] C:\Users\heber\Documents\Compressor\264.mkv
-> [全] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.AVC.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
mkdir C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.AVC.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv" -D ( "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.

In [18]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)
None

In [24]:
get_captures("x264", x264_filename)
generate_videoinfo(f'{x264_foldername}\\{x264_filename}',"x264.log")

Capture x264
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome
RELEASE DATE..............: 2021-01-21
RELEASE SiZE..............: 17.69 GB
SOURCE....................: 

.Media.Info
RUNTiME...................: 2 h 22 min
ViDEO CODEC...............: x264 High@L4.1 @ 13.35Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 23.976
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: English MLP FBA 8 @ 4271.44Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs&eng)
SUBTiTLES.................: S_HDMV/PGS : Chinese(cht&eng)
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(cht)
SUBTiTLES.................: S_HDMV/PGS : English(eng)


.x264.Info
[/quote]


'[quote][font=Lucida Console]\n.Release.Info\nENCODER...................: PTH\nRELEASE NAME..............: Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome\nRELEASE DATE..............: 2021-01-21\nRELEASE SiZE..............: 17.69 GB\nSOURCE....................: \n\n.Media.Info\nRUNTiME...................: 2 h 22 min\nViDEO CODEC...............: x264 High@L4.1 @ 13.35Mb/s\nRESOLUTiON................: 1920x800\nDiSPLAY ASPECT RATiO......: 2.400\nFRAME RATE................: 23.976\nBIT DEPTH.................: 8 bits\nAUDiO CODEC...............: English MLP FBA 8 @ 4271.44Kb/s \n \nSUBTiTLES.................: S_HDMV/PGS : Chinese(chs&eng)\nSUBTiTLES.................: S_HDMV/PGS : Chinese(cht&eng)\nSUBTiTLES.................: S_HDMV/PGS : Chinese(chs)\nSUBTiTLES.................: S_HDMV/PGS : Chinese(cht)\nSUBTiTLES.................: S_HDMV/PGS : English(eng)\n\n\n.x264.Info\n[/quote]'

## x265

In [26]:
x265_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX.AVC", "x265.10bit")
x265_foldername=PROCUCT_DEST_DIR+"\\"+x265_filename.replace(".mkv",'')
NEWMOVIE=f'{x265_foldername}\\{x265_filename}'
x265_fullpath=NEWMOVIE
cmd=f'mkdir {x265_foldername} ^& '
cmd+=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\265.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\265.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

即将执行: 
   [剩] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
 + [视] C:\Users\heber\Documents\Compressor\265.mkv
-> [全] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome.mkv
mkdir C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome.mkv" -D ( "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.Tru

In [11]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)

CompletedProcess(args='start cmd /c mkdir C:\\Users\\heber\\Music\\Get.Out.2017.UHD.Blu-Ray.2160p.x265.10bit.DTS-louco@PTH ^& C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvmerge.exe -o "C:\\Users\\heber\\Music\\Get.Out.2017.UHD.Blu-Ray.2160p.x265.10bit.DTS-louco@PTH\\Get.Out.2017.UHD.Blu-Ray.2160p.x265.10bit.DTS-louco@PTH.mkv" -D ( "C:\\Users\\heber\\Music\\Get.Out.2017.UHD.Blu-Ray.2160p.REMUX.DTS-louco@PTH.mkv" )    -A -S -T -M -B --no-chapters ( "C:\\Users\\heber\\Documents\\Compressor\\265.mkv" ) ^& pause', returncode=0)

In [27]:
generate_videoinfo(x265_fullpath,"x265.log")
print(x265_filename)
get_captures("x265", x265_fullpath)

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome
RELEASE DATE..............: 2021-01-21
RELEASE SiZE..............: 16.885 GB
SOURCE....................: 

.Media.Info
RUNTiME...................: 2 h 22 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 12.54Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 23.976
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: English MLP FBA 8 @ 4271.44Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs&eng)
SUBTiTLES.................: S_HDMV/PGS : Chinese(cht&eng)
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(cht)
SUBTiTLES.................: S_HDMV/PGS : English(eng)


.x265.Info
x265 [info]: Main 10 profile, Level-5.1 (High tier)
x265 [info]: frame I:   1505, Avg QP:17.73  

# Step5 发布

In [28]:
# https://api.rhilip.info/ptgen.html

def get_bbcode(file, src="",logname="", title=None):
    print("Title:", file.replace("."," "))
#     由ZERO13@PTHome字幕组完成校对、调整。
    picbed='''
[url=https://sm.ms/image/s5faT6HditZRAmz][img]https://i.loli.net/2021/01/21/s5faT6HditZRAmz.jpg[/img][/url]
[url=https://sm.ms/image/FowetS7KdhRJAnX][img]https://i.loli.net/2021/01/21/FowetS7KdhRJAnX.jpg[/img][/url]
[url=https://sm.ms/image/Mkl9qzc3mn7ZeRi][img]https://i.loli.net/2021/01/21/Mkl9qzc3mn7ZeRi.jpg[/img][/url]
[url=https://sm.ms/image/LW5bq8ofTm1FrDg][img]https://i.loli.net/2021/01/21/LW5bq8ofTm1FrDg.jpg[/img][/url]
[url=https://sm.ms/image/eJrCinxz9G7Dh4P][img]https://i.loli.net/2021/01/21/eJrCinxz9G7Dh4P.jpg[/img][/url]
[url=https://sm.ms/image/U5YzARxu4qWiMtg][img]https://i.loli.net/2021/01/21/U5YzARxu4qWiMtg.jpg[/img][/url]
    '''
    if "REMUX" in file:
        picbed='''
[url=https://sm.ms/image/DXSGehZUtavp51n][img]https://i.loli.net/2021/01/21/DXSGehZUtavp51n.jpg[/img][/url]
[url=https://sm.ms/image/5tGfhgXBnrQCO1b][img]https://i.loli.net/2021/01/21/5tGfhgXBnrQCO1b.jpg[/img][/url]
[url=https://sm.ms/image/13UCdleYK7vAioP][img]https://i.loli.net/2021/01/21/13UCdleYK7vAioP.jpg[/img][/url]
[url=https://sm.ms/image/gFnQiyUTY4uH1Xj][img]https://i.loli.net/2021/01/21/gFnQiyUTY4uH1Xj.jpg[/img][/url]
[url=https://sm.ms/image/bpewl6C1vnYX5cF][img]https://i.loli.net/2021/01/21/bpewl6C1vnYX5cF.jpg[/img][/url]
[url=https://sm.ms/image/VpQHfztGyUBN2jX][img]https://i.loli.net/2021/01/21/VpQHfztGyUBN2jX.jpg[/img][/url]
    '''
    bbcode='''
[quote][size=5][color=RoyalBlue]原盘来自TAiCHi，字幕素材来自字幕库
字幕转载务必注意礼节，请保留原作者信息，谢绝二次提取修改！
在此感谢各位原作者及分享者！如有侵权请联系删除！[/color][/size][/quote]

[img]https://pic.imgdb.cn/item/6009723a3ffa7d37b37938c8.jpg[/img]

◎译　　名　阿甘正传 / 福雷斯特·冈普
◎片　　名　Forrest Gump
◎年　　代　1994
◎产　　地　美国
◎类　　别　剧情 / 爱情
◎语　　言　英语
◎上映日期　1994-06-23(洛杉矶首映) / 1994-07-06(美国)
◎IMDb评分  8.8/10 from 1762374 users
◎IMDb链接  https://www.imdb.com/title/tt0109830
◎豆瓣评分　9.5/10 from 1623538 users
◎豆瓣链接　https://movie.douban.com/subject/1292720/
◎片　　长　142分钟
◎导　　演　罗伯特·泽米吉斯 Robert Zemeckis
◎编　　剧　艾瑞克·罗斯 Eric Roth / 温斯顿·格鲁姆 Winston Groom
◎主　　演　汤姆·汉克斯 Tom Hanks
　　　　　  罗宾·怀特 Robin Wright
　　　　　  加里·西尼斯 Gary Sinise
　　　　　  麦凯尔泰·威廉逊 Mykelti Williamson
　　　　　  莎莉·菲尔德 Sally Field
　　　　　  海利·乔·奥斯蒙 Haley Joel Osment
　　　　　  迈克尔·康纳·亨弗里斯 Michael Conner Humphreys
　　　　　  哈罗德·G·赫瑟姆 Harold G. Herthum
　　　　　  山姆·安德森 Sam Anderson
　　　　　  伊俄涅·M·特雷奇 Ione M. Telech
　　　　　  彼得·道博森 Peter Dobson
　　　　　  希芳·法隆 Siobhan Fallon
　　　　　  伊丽莎白·汉克斯 Elizabeth Hanks
　　　　　  汉娜·豪尔 Hanna Hall
　　　　　  克里斯托弗·琼斯 Christopher Jones
　　　　　  罗布·兰德里 Rob Landry
　　　　　  杰森·麦克奎尔 Jason McGuire
　　　　　  桑尼·施罗耶 Sonny Shroyer
　　　　　  艾德·戴维斯 Ed Davis
　　　　　  丹尼尔C.斯瑞派克 Daniel C. Striepeke
　　　　　  大卫·布里斯宾 David Brisbin
　　　　　  德博拉·麦克蒂尔 Deborah McTeer
　　　　　  艾尔·哈林顿 Al Harrington
　　　　　  阿非莫·奥米拉 Afemo Omilami
　　　　　  约翰·沃德斯塔德 John Voldstad
　　　　　  迈克尔·伯吉斯 Michael Burgess
　　　　　  埃里克·安德伍德 Eric Underwood
　　　　　  拜伦·明斯 Byron Minns
　　　　　  斯蒂芬·布吉格沃特 Stephen Bridgewater
　　　　　  约翰·威廉·高尔特 John William Galt
　　　　　  希拉里·沙普兰 Hilary Chaplain
　　　　　  伊莎贝尔·罗斯 Isabel Rose
　　　　　  理查德·达历山德罗 Richard D&#39;Alessandro
　　　　　  迪克·史迪威 Dick Stilwell
　　　　　  迈克尔-杰斯 Michael Jace
　　　　　  杰弗里·布莱克 Geoffrey Blake
　　　　　  瓦妮莎·罗斯 Vanessa Roth
　　　　　  迪克·卡维特 Dick Cavett
　　　　　  马拉·苏查雷特扎 Marla Sucharetza
　　　　　  乔·阿拉斯奇 Joe Alaskey
　　　　　  W·本森·泰瑞 W. Benson Terry
　　　　　  约翰·列侬 John Lennon
　　　　　  埃尔维斯·普雷斯利 Elvis Presley
　　　　　  罗纳德·里根 Ronald Reagan
　　　　　  鲍勃·霍普 Bob Hope
　　　　　  约翰·肯尼迪 John F. Kennedy
　　　　　  理查德·尼克松 Richard Nixon
　　　　　  吴俊全 Junquan Wu
　　　　　  蒂莫西·麦克尼尔 Timothy McNeil
　　　　　  杰拉尔德·福特 Gerald Ford
　　　　　  布雷特·赖斯 Brett Rice


◎标　　签　励志 | 经典 | 人生 | 成长 | 美国 | 信念 | 剧情 | 人性

◎简　　介  

　　公汽车站，一位跑鞋破旧但干净的中年人（汤姆·汉克斯 饰）回忆起了自己的第一双鞋。少时的 Forrest Gump（迈克·康纳·汉弗莱 饰）因身体原因必须戴着腿部支架行走，加上智商只有75，入学后免不了被欺负。在同学 Jenny Curran（汉娜·豪尔 饰）的提醒下，Forrest 奔跑起来。奇迹发生了，支架散落后，Forrest 跑得像风一样快。长大后，Forrest 更是将天赋发挥到了极致。他成了橄榄球星、大学毕业生、战争英雄、外交使者、捕虾船长、慈善家。他目睹了战友 Benjamin Buford Blue（梅克尔蒂·威廉森 饰）阵亡，Dan Taylor 中尉（加里·辛尼斯 饰）与上帝和解，母亲（萨莉·菲尔德 饰）病逝。而当夜晚来临，他总会思念 Jenny（罗宾·赖特 饰）……

◎获奖情况  

　　第67届奥斯卡金像奖(1995)
　　最佳影片 史蒂夫·蒂彻 / 温迪·芬德曼 / 史蒂夫·斯达克
　　最佳导演 罗伯特·泽米吉斯
　　最佳男主角 汤姆·汉克斯
　　最佳男配角(提名) 加里·西尼斯
　　最佳改编剧本 艾瑞克·罗斯
　　最佳摄影(提名) 唐·伯吉斯
　　最佳剪辑 阿瑟·施密特
　　最佳视觉效果 斯蒂芬·罗森包姆 / 肯·拉斯顿 / 乔治·墨菲 / 艾伦·霍尔
　　最佳音效剪辑(提名) 兰迪·汤姆 / 格洛里亚·S·博德尔斯
　　最佳音响(提名) 兰迪·汤姆 / 威廉·B·卡普兰 / 丹尼斯·S·桑兹 / 汤姆·约翰逊
　　最佳艺术指导(提名) 南茜·黑格 / 瑞克·卡特
　　最佳化妆(提名) 丹尼尔C.斯瑞派克 / 霍利·达莫尔 / 朱迪斯·A·科里
　　最佳原创配乐(提名) 亚伦·史维斯查

　　第48届英国电影学院奖(1995)
　　电影奖 最佳影片(提名) 温迪·芬德曼 / 史蒂夫·斯达克 / 史蒂夫·蒂彻 / 罗伯特·泽米吉斯
　　电影奖 最佳导演(提名) 罗伯特·泽米吉斯
　　电影奖 最佳男主角(提名) 汤姆·汉克斯
　　电影奖 最佳女配角(提名) 莎莉·菲尔德
　　电影奖 最佳改编剧本(提名) 艾瑞克·罗斯
　　电影奖 最佳摄影(提名) 唐·伯吉斯
　　电影奖 最佳剪辑(提名) 阿瑟·施密特
　　电影奖 最佳特殊视觉效果 江道格 / 肯·拉斯顿 / 斯蒂芬·罗森包姆 / 乔治·墨菲 / 艾伦·霍尔

　　第19届日本电影学院奖(1996)
　　最佳外语片(提名)

　　第16届中国电影金鸡奖(1996)
　　最佳外国影片译制奖

[img]https://images2.imgbox.com/04/6b/Ggp5ReQb_o.png[/img]



'''+f'{generate_videoinfo(file,logname,src=src)}\n{picbed}\n{get_mediainfo(file)}'
    pyperclip.copy(bbcode)
SRC=r"Get Out 2017 UHD Blu-Ray 2160p&1080p HEVC&AVC DTS-X 7.1&5.1-CHDBits".replace(" ",".")

In [7]:
from utils import *
REMUX=r"C:\Users\heber\Music\Get.Out.2017.UHD.Blu-Ray.2160p.HEVC.REMUX.DTS-HD.M.A.-louco@PTH\Get.Out.2017.UHD.Blu-Ray.2160p.HEVC.REMUX.DTS-HD.M.A-louco@PTH.mkv"
get_bbcode(REMUX,src=SRC)

Title: C:\Users\heber\Music\Get Out 2017 UHD Blu-Ray 2160p HEVC REMUX DTS-HD M A -louco@PTH\Get Out 2017 UHD Blu-Ray 2160p HEVC REMUX DTS-HD M A-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Get.Out.2017.UHD.Blu-Ray.2160p.HEVC.REMUX.DTS-HD.M.A-louco@PTH
RELEASE DATE..............: 2021-01-21
RELEASE SiZE..............: 53.345 GB
SOURCE....................: Get.Out.2017.UHD.Blu-Ray.2160p&1080p.HEVC&AVC.DTS-X.7.1&5.1-CHDBits

.Media.Info
RUNTiME...................: 1 h 44 min
ViDEO CODEC...............: ? Main 10@L5.1@High @ 67.84Mb/s
RESOLUTiON................: 3840x2160
DiSPLAY ASPECT RATiO......: 1.778
FRAME RATE................: 23.976
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: English DTS-HD MA 8 @ 5476.73Kb/s 
 
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs)
SUBTiTLES...

In [47]:
X264FINAL=r"C:\Users\heber\Music\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH.mkv"
get_bbcode(X264FINAL,src=SRC,logname="x264.log")

Title: C:\Users\heber\Music\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p x264 DTS-louco@PTH\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p x264 DTS-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH
RELEASE DATE..............: 2021-01-06
RELEASE SiZE..............: 11.457 GB
SOURCE....................: A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p AVC DTS-HD 5.1-hdsng@HDSky

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: x264 High@L4.1 @ 16.47Mb/s
RESOLUTiON................: 1920x1036
DiSPLAY ASPECT RATiO......: 1.853
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Russian DTS 5.1 @ 768.0Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(zhs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zht)
SUBTiTLES.................: S_HDMV/PGS : Russian(ru)


.x264.Inf

In [15]:
get_bbcode(x265_fullpath,src=SRC,logname="x265.log")

Title: C:\Users\heber\Music\Get Out 2017 UHD Blu-Ray 2160p x265 10bit DTS-louco@PTH\Get Out 2017 UHD Blu-Ray 2160p x265 10bit DTS-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Get.Out.2017.UHD.Blu-Ray.2160p.x265.10bit.DTS-louco@PTH
RELEASE DATE..............: 2021-01-21
RELEASE SiZE..............: 4.867 GB
SOURCE....................: Get.Out.2017.UHD.Blu-Ray.2160p&1080p.HEVC&AVC.DTS-X.7.1&5.1-CHDBits

.Media.Info
RUNTiME...................: 1 h 44 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 5.13Mb/s
RESOLUTiON................: 3840x1600
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 23.976
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: English DTS 5.1 @ 1509.0Kb/s 
 
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs)
SUBTiTLES.................: S_TEXT/ASS

# 备忘录